# EMIT L2A Reflectance – end‑to‑end example
This notebook follows the same **flow** as your original: setup → search → pick → download → open/flatten → mask → (optional) orthorectify → visualize → (optional) export/merge.

It uses the refactored helpers in `emitref/` plus your original `emit_tools` (vendored in the package).

## 1) Environment & imports

In [ ]:
! pip install -r requirements.txt

In [ ]:
# If you unzipped the package next to this notebook, ensure Python can import it.
# Adjust the path below if needed.
import sys, pathlib
p = pathlib.Path().resolve()
# Example: sys.path.append(str(p / "emitref_refactor_with_emit_tools"))
# If you've installed as a package (pip -e .), you can skip sys.path fiddling.

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# interactive (optional)
import hvplot.xarray  # noqa: F401
import holoviews as hv
hv.extension('bokeh')

from datetime import datetime, timezone, timedelta

# Our helpers
from EMIT_data.EMIT_utils import (
login, search, choose_nearest, download_reflectance,
open_reflectance, attach_wavelengths
)
from EMIT_data.geo import get_pixel_center_coords, apply_glt, ortho_xr
from EMIT_data.viz import plot_band_nm, rgb_from_nm, hv_quicklook
from EMIT_data.emit_tools import (
    quality_mask, band_mask,
    spatial_subset,
    ortho_xr 
)


In [ ]:
LAT = 50.3794
LON = 43.1656
DATE = "2024-06-04"
DAYS_BEFORE = 5
DAYS_AFTER  = 5
SEARCH_BUFFER_M = 500000    

target_dt = datetime(2024, 6, 4, tzinfo=timezone.utc)  # center date
start = target_dt - timedelta(days=DAYS_BEFORE)
end   = target_dt + timedelta(days=DAYS_AFTER)

print(start, "→", end)


In [ ]:
# This opens a browser/device-code flow on first run and caches your token (persist=True)
login(persist=True)


## 4) Search EMIT L2A reflectance and pick nearest granule

In [ ]:
picks = search(point=(LON, LAT), buffer_m=SEARCH_BUFFER_M, start=None, end=None, cloud_cover = (0.0,100.0))

In [ ]:
picks = search(point=(LON, LAT), buffer_m=SEARCH_BUFFER_M, start=None, end=None, cloud_cover = (0.0,0.01))

In [ ]:
picks[0]["umm"]["CloudCover"]

In [ ]:
picks[0].keys()

In [ ]:
picks[0]["umm"]["ProviderDates"][0]["Date"]

In [ ]:
pick = choose_nearest(picks, target_dt)
if pick is None:
    raise SystemExit("No EMIT granules found in the window.")
pick

## 5) Download reflectance (NetCDF)

In [ ]:
from pathlib import Path
out_dir = Path("data/emit")
paths = download_reflectance(pick, out_dir, assets=['_RFL_'])
paths

In [ ]:
ds = open_reflectance(paths[0], engine='h5netcdf')      
ds = attach_wavelengths(ds)           
ds


In [ ]:
mask_paths = download_reflectance(pick, out_dir, assets=['_MASK_'])

In [ ]:
from EMIT_data.emit_tools import emit_xarray

In [ ]:
qmask = quality_mask(mask_paths[0], quality_bands=[0,1,2,3,4])
bmask = band_mask(mask_paths[0])  

ds_emit = emit_xarray(paths[0], ortho=True, engine="netcdf4")

In [ ]:
ds_emit['wavelengths'].values.shape

In [ ]:
ds_vis = ds_emit


In [ ]:
rfl_var = "reflectance" if "reflectance" in ds_vis.data_vars else list(ds_vis.data_vars)[0]
ds_vis[rfl_var].dims

In [ ]:
from EMIT_data.viz import hv_quicklook_nearest_valid, emit_quicklook_matplotlib

In [ ]:
import hvplot.xarray
import jupyter_bokeh
import holoviews as hv, panel as pn
pn.extension(comms='colab')
hv.extension('bokeh')

In [ ]:
emit_quicklook_matplotlib(ds_vis, white_balance=False, gamma = 1)


In [ ]:
import os

In [ ]:
# !pip install brain_pipe

In [ ]:
# !pip uninstall -y hytools

In [ ]:
# !pip install hy-tools-lite

In [ ]:
!pip install "git+https://github.com/EnSpec/hytools.git"

In [ ]:
from EMIT_data.emit_proj import nc_to_envi

In [ ]:
emit_rfl_nc = "data/emit/EMIT_L2A_RFL_001_20240529T123132_2415008_008.nc"      # or EMIT L1B radiance
emit_obs_nc = "data/emit/EMIT_L2A_MASK_001_20240529T123132_2415008_008.nc"      # optional
s2_tif_path = "data/s2/S2A_37UGR_20230417_0_L2A_visual.tif"       # your Sentinel-2 GeoTIFF
out_dir      = "emit_out"             # or any writable folder
tmp_dir      = "emit_tmp"
os.makedirs(out_dir, exist_ok=True)
os.makedirs(tmp_dir, exist_ok=True)

# Run: match CRS to S2, keep EMIT at 60 m (set match_res=True to copy S2 pixel size)
nc_to_envi(
    img_file=emit_rfl_nc,
    out_dir=out_dir,
    temp_dir=tmp_dir,
    obs_file=emit_obs_nc,     # or None
    export_loc=True,          # also produce *_LOC
    crid="000",
    s2_tif_path=s2_tif_path,  # copy CRS from S2
    match_res=False,          # True → resample to S2 pixel size
    write_xml=True            # write XML sidecars
)

In [ ]:
from EMIT_data.viz import show_emit_rgb_from_envi

In [ ]:
out_dir = "emit_out"
show_emit_rgb_from_envi(out_dir)

In [ ]:
emit_quicklook_matplotlib(ds_vis, white_balance=False, gamma = 1)